[View in Colaboratory](https://colab.research.google.com/github/joaoflf/ml-playground/blob/master/mnist_alexnet.ipynb)

### Application of a Alexnet inspired ConvNet to MNIST

In [26]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

n_iters = 10000
batch_size= 100
learning_rate = 0.001

n_inputs = 784 # MNIST data input (img shape: 28*28)
n_classes = 10

tf.reset_default_graph()

weight_dict = {
    'wc1': tf.Variable(tf.random_normal([3, 3, 1, 64])),
    'wc2': tf.Variable(tf.random_normal([3, 3, 64, 128])),
    'wc3': tf.Variable(tf.random_normal([3, 3, 128, 256])),
    'wd1': tf.Variable(tf.random_normal([4*4*256, 1024])),
    'wd2': tf.Variable(tf.random_normal([1024, 1024])),
    'out': tf.Variable(tf.random_normal([1024, 10]))
}
bias_dict = {
    'bc1': tf.Variable(tf.random_normal([64])),
    'bc2': tf.Variable(tf.random_normal([128])),
    'bc3': tf.Variable(tf.random_normal([256])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'bd2': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


X = tf.placeholder(tf.float32, [None, n_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

def conv(name, input_layer, w, b):
  return tf.nn.relu(tf.nn.conv2d(input_layer, w, [1, 1, 1, 1], padding='SAME')+ b, name=name)

def max_pool(name, input_layer, k, s):
  return tf.nn.max_pool(input_layer, [1, k, k, 1], [1, s, s, 1], padding='SAME', name= name)

def norm(name, input_layer, l_size=4):
  return tf.nn.lrn(input_layer, l_size, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name=name)

def conv_net(inputs, weights, biases):
  # Reshape input picture
  inputs = tf.reshape(inputs, shape=[-1, 28, 28, 1])
  
  conv1 = conv('conv1', inputs, weights['wc1'], biases['bc1'])
  max_pool1 = max_pool('max_pool1', conv1, 2, 2)
  norm1 = norm('norm1', max_pool1)
  dropout1 = tf.nn.dropout(norm1, keep_prob)
  
  conv2 = conv('conv2', dropout1, weights['wc2'], biases['bc2'])
  max_pool2 = max_pool('max_pool2', conv2, 2, 2)
  norm2 = norm('norm2', max_pool2)
  dropout2 = tf.nn.dropout(norm2, keep_prob)
  
  conv3 = conv('conv3', dropout2, weights['wc3'], biases['bc3'])
  max_pool3 = max_pool('max_pool3', conv3, 2, 2)
  norm3 = norm('norm3', max_pool3)
  dropout3 = tf.nn.dropout(norm3, keep_prob, name='dropout3')
  
  dense_input = tf.reshape(dropout3, [-1, 4096])
  fc1 = tf.nn.relu(tf.matmul(dense_input, weights['wd1']) + biases['bd1'], name='fc1')
  fc2 = tf.nn.relu(tf.matmul(fc1, weights['wd2']) + biases['bd2'], name='fc2')
  out = tf.matmul(fc2, weights['out']) + biases['out']
  
  return out

y_pred = conv_net(X, weight_dict, bias_dict)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels= y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.initialize_all_variables()

with tf.Session() as sess:
  sess.run(init)
  
  for i in range(n_iters):
    X_batch, y_batch = mnist.train.next_batch(batch_size)
    sess.run(optimizer, feed_dict={X: X_batch, y: y_batch, keep_prob:0.8})
    
    if i % 1000 == 0:
      loss, acc = sess.run([cost, accuracy], feed_dict={X: X_batch, y: y_batch, keep_prob:1.})
      print("Iter " + str(i) + ", Minibatch Loss= " + "{:.3f}".format(loss) + ", Training Accuracy= " + "{:.3f}".format(acc))
    
  print('Done!')
  print("Testing Accuracy:", sess.run(accuracy, feed_dict={X: mnist.test.images, y: mnist.test.labels, keep_prob: 1.}))



Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Iter 0, Minibatch Loss= 307048.125000, Training Accuracy= 0.17000
Iter 1000, Minibatch Loss= 1800.776855, Training Accuracy= 0.93000
Iter 2000, Minibatch Loss= 3769.105713, Training Accuracy= 0.92000
Iter 3000, Minibatch Loss= 1387.415283, Training Accuracy= 0.95000
Iter 4000, Minibatch Loss= 412.688995, Training Accuracy= 0.95000
Iter 5000, Minibatch Loss= 1074.801514, Training Accuracy= 0.95000
Iter 6000, Minibatch Loss= 311.232727, Training Accuracy= 0.96000
Iter 7000, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 8000, Minibatch Loss= 170.173599, Training Accuracy= 0.97000
Iter 9000, Minibatch Loss= 101.374847, Training Accuracy= 0.98000
Done!
Testing Accuracy: 0.9834
